In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing, feature_extraction
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [ ]:
#Chargement de datasets
features = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","classification","difficulty"]# nom des variables

train = pd.read_csv ('/content/drive/MyDrive/dataset/KDDTrain+.csv', header=None, names=features)#lecture dataset d'apprentissage
test = pd.read_csv ('/content/drive/MyDrive/dataset/KDDTest+.csv', header=None, names=features)#lecture dataset de test
train.drop("difficulty", axis=1, inplace=True)
test.drop("difficulty", axis=1, inplace=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Remplacement de tous les différents noms d'attaque par leur type d'attaque
#train dataset
train.loc[(train['classification'] =='smurf')|(train['classification'] =='neptune') | (train['classification'] =='back') | (train['classification'] =='teardrop') |(train['classification'] =='pod')| (train['classification']=='land'),'classification'] = 'Dos'
train.loc[(train['classification'] =='satan')|(train['classification'] =='ipsweep') | (train['classification'] =='portsweep') | (train['classification'] =='nmap'),'classification'] = 'probe'
train.loc[(train['classification'] =='spy')|(train['classification'] =='phf')|(train['classification'] =='multihop')|(train['classification'] =='ftp_write') | (train['classification'] =='imap') | (train['classification'] =='warezmaster') |(train['classification'] =='guess_passwd')| (train['classification']=='warezclient'),'classification'] = 'r2l'
train.loc[(train['classification'] =='buffer_overflow')|(train['classification'] =='rootkit') | (train['classification'] =='loadmodule') | (train['classification'] =='perl'),'classification']='u2r'
train.loc[(train['classification'] =='normal'),'classification'] = 'normal'

#test dataset
test.loc[(test['classification'] =='smurf')|(test['classification'] =='neptune') | (test['classification'] =='back') | (test['classification'] =='teardrop') |(test['classification'] =='pod')| (test['classification']=='land')|(test['classification']=='apache2')|(test['classification']=='udpstorm')|(test['classification']=='processtable')|(test['classification']=='mailbomb'),'classification'] = 'Dos'
test.loc[(test['classification'] =='guess_passwd')|(test['classification'] =='ftp_write')|(test['classification'] =='imap')|(test['classification'] =='phf') | (test['classification'] =='multihop') | (test['classification'] =='warezmaster') |(test['classification'] =='snmpgetattack')| (test['classification']=='named')|(test['classification'] =='xlock')|(test['classification'] =='xsnoop')|(test['classification'] =='sendmail')|(test['classification'] =='httptunnel')|(test['classification'] =='worm')|(test['classification'] =='snmpguess'),'classification'] = 'r2l'
test.loc[(test['classification'] =='satan')|(test['classification'] =='ipsweep') | (test['classification'] =='portsweep') | (test['classification'] =='nmap') | (test['classification'] =='saint')|(test['classification'] =='mscan'),'classification'] = 'probe'
test.loc[(test['classification'] =='buffer_overflow')|(test['classification'] =='rootkit') | (test['classification'] =='loadmodule') | (test['classification'] =='xterm')|(test['classification'] =='sqlattack')|(test['classification'] =='ps')|(test['classification'] =='perl'),'classification']='u2r'
test.loc[(test['classification'] =='normal'),'classification'] = 'normal'

In [ ]:
print("-----------------amélioration du dataset --------------------")

df1 = pd.DataFrame()
df2 = pd.DataFrame()
for x in range(train.shape[0]):
    if train.iloc[x].classification == "u2r":
        df1 = df1.append(train.iloc[x])
    if train.iloc[x].classification == "r2l":
        df2 = df2.append(train.iloc[x])
for i in range(150):
  train = train.append(df1)
for i in range(30):
  train = train.append(df2)

-----------------amélioration du dataset --------------------


KeyboardInterrupt: ignored

In [ ]:
#section qui stock chaque valeur distinctes d'une colonne catégoriale dans une liste précedé par 'nomcolonne_'
#train dataset
colonnes_nouv_train = []# liste qui contient toute les nouvelles colonnes pour le dataset d'apprentissage

protocol_distincts = sorted(train.protocol_type.unique())# obtenir les valeur distinctes de la colonnes protocol_type
for e in protocol_distincts :#ajout dans la liste
    colonnes_nouv_train = colonnes_nouv_train + ["Protocole_type_"+ e]

service_distincts = sorted(train.service.unique())# obtenir les valeur distinctes de la colonnes service_type
for e in service_distincts :#ajout dans la liste
    colonnes_nouv_train = colonnes_nouv_train + ["service_type_"+ e]

flag_distincts = sorted(train.flag.unique())# obtenir les valeur distinctes de la colonnes flag_type
for e in flag_distincts :#ajout dans la liste
    colonnes_nouv_train = colonnes_nouv_train + ["flag_type_"+ e]

#test dataset

colonnes_nouv_test = []# liste qui contient toute les nouvelles colonnes pour le dataset de test

protocol_distincts = sorted(test.protocol_type.unique())# obtenir les valeur distinctes de la colonnes protocol_type
for e in protocol_distincts :#ajout dans la liste
    colonnes_nouv_test = colonnes_nouv_test + ["Protocole_type_"+ e]

service_distincts = sorted(test.service.unique())# obtenir les valeur distinctes de la colonnes service_type
for e in service_distincts :#ajout dans la liste
    colonnes_nouv_test = colonnes_nouv_test + ["service_type_"+ e]

flag_distincts = sorted(test.flag.unique())# obtenir les valeur distinctes de la colonnes flag_type
for e in flag_distincts :#ajout dans la liste
    colonnes_nouv_test = colonnes_nouv_test + ["flag_type_"+ e]


# obtenir les colonnes catégoriales avec les valeurs d'origine
train_colonnes_catégoriales = train[['protocol_type', 'service', 'flag']]
test_colonnes_catégoriales = test[['protocol_type', 'service', 'flag']]


In [ ]:
"""---------------------------------- One-Hot-Encoder ----------------------------------"""
print("One-Hot-Encoder ...")

#transformation des données catégoriales vers des données numérique
enc = OneHotEncoder()#instanciation
train_Hot_Encoder = enc.fit_transform(train_colonnes_catégoriales)
train_Hot_Encoder = pd.DataFrame(train_Hot_Encoder.toarray(), columns = colonnes_nouv_train)
#refaire la meme chose pour le dataset de test
test_Hot_Encoder = enc.fit_transform(test_colonnes_catégoriales)
test_Hot_Encoder = pd.DataFrame(test_Hot_Encoder.toarray(), columns = colonnes_nouv_test)


#nouveau dataset
train_One_Hot_Encoder = train.join(train_Hot_Encoder)
train_One_Hot_Encoder.drop('flag',axis=1, inplace=True)
train_One_Hot_Encoder.drop('protocol_type',axis=1, inplace=True)
train_One_Hot_Encoder.drop('service',axis=1, inplace=True)
train_One_Hot_Encoder.to_csv(r"/content/drive/MyDrive/dataset/new dataset/OneHotEncoder_train.csv",index = False) #sauvgarder le fichier

#refaire la meme chose pour le dataset du test
test_One_Hot_Encoder = test.join(test_Hot_Encoder)
test_One_Hot_Encoder.drop('flag',axis=1, inplace=True)
test_One_Hot_Encoder.drop('protocol_type',axis=1, inplace=True)
test_One_Hot_Encoder.drop('service',axis=1, inplace=True)
#l'ajout des 6 colonnes manquantes dans le dataset de test
service_train = train['service'].tolist()
service_test = test['service'].tolist()
diff = list(set(service_train) - set(service_test))
for col in diff:
    test_One_Hot_Encoder['service_'+col] = 0
test_One_Hot_Encoder.to_csv(r"/content/drive/MyDrive/dataset/new dataset/OneHotEncoder_test.csv",index = False) #sauvgarder le fichier


In [ ]:
"""---------------------------------- Standardisation des données ----------------------------------"""
print("Standardisation des données ...")

#diviser les données entre X et Y
#train
X_train = train_One_Hot_Encoder.drop('classification', 1)
Y_train = train_One_Hot_Encoder.classification
#test
X_test = test_One_Hot_Encoder.drop('classification', 1)
Y_test = test_One_Hot_Encoder.classification

#StandardScaler
#on doit sauvgarder la premiére colonne
premiére_ligne_train = list(X_train)
premiére_ligne_test = list(X_test)
#train
X_train = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = premiére_ligne_train
X_train.to_csv(r"/content/drive/MyDrive/dataset/new dataset/duplicated/OneHotEncoder_scaled_train.csv",index = False)
Y_train.to_csv(r"/content/drive/MyDrive/dataset/new dataset/duplicated/Y_train.csv",index = False)

#test
X_test = StandardScaler().fit_transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = premiére_ligne_test

X_test.to_csv(r"/content/drive/MyDrive/dataset/new dataset/duplicated/OneHotEncoder_scaled_test.csv",index = False)
Y_test.to_csv(r"/content/drive/MyDrive/dataset/new dataset/duplicated/Y_test.csv",index = False)


In [ ]:
"""---------------------------------- Selection des variables ---------------------------------"""

#Feature Gain-Mutual Information
#affichier le plot des variables selon l'algorithme mutual information gain
print("Selection des variables ...")
feature_scores = mutual_info_classif(X_train,Y_train)
feature_scores = pd.Series(feature_scores)
feature_scores.index = premiére_ligne_train
print("variables trié selon l'importance :")
print(feature_scores.sort_values(ascending = False))
plt.figure(figsize = (30, 10))
bar = feature_scores.sort_values(ascending = False).plot(kind="bar")
bar.set_title("l'information mutuelle entre les variables indépendantes et la variable dépendante")
bar.set_xlabel("Variables")
bar.set_ylabel("information mutuelle entre la varabie X et la variable 'classification'")

#séléctionner les N premiére variable
selection = SelectKBest(mutual_info_classif, k=35)
selection.fit(X_train,Y_train)

print("les variables choisie par l'algorithme information gain :")
print(X_train.columns[selection.get_support()])

